# Installs

Installing libraries 1by1 due to some compatibility issues encountered on Windows

In [1]:
!pip3 install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
!pip install -q transformers

In [3]:
!pip install -q trl

In [4]:
!pip install -q pandas

In [5]:
!pip install -q datasets

In [6]:
!pip install -q huggingface_hub

In [7]:
!pip install -q bitsandbytes

In [8]:
!pip install -q peft

In [9]:
!pip install -q accelerate

In [11]:
!pip install -q setuptools

In [12]:
!pip install -q ipywidgets

In [14]:
!pip install -q python-dotenv

In [34]:
!pip install -q tensorboard

## Freeze requirements

In [43]:
!pip freeze > requirements.txt

# Libs Params

In [1]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")
#MODEL_NAME = "NousResearch/Llama-2-7b-chat-hf"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

Device: cuda:0


In [2]:
OUTPUT_DIR = "experiments"

# HugFace login

In [3]:
from dotenv import load_dotenv
import os
from huggingface_hub import HfApi
load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")
if hf_token is None:
    raise ValueError("Hugging Face token not found. Please check your .env file.")

api = HfApi()
user_info = api.whoami(token=hf_token)
print("Logged in as:", user_info['name'])

Logged in as: bnalyv


# Dataset

In [4]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

# Functions

In [5]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()


def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
{summary}
""".strip()

In [6]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    return re.sub(r"\^[^ ]+", "", text)


def create_conversation_text(data_point):
    text = ""
    for item in data_point["log"]:
        user = clean_text(item["user utterance"])
        text += f"user: {user.strip()}\n"

        agent = clean_text(item["system response"])
        text += f"agent: {agent.strip()}\n"

    return text

In [7]:
def generate_text(data_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)

    conversation_text = create_conversation_text(data_point)
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text, summary),
    }

In [8]:
example = generate_text(dataset["train"][0])
print(example["summary"])
print("\n ############## \n")
print(example["conversation"])
print("\n ############## \n")
print(example["text"])

Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.

 ############## 

user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user: My iPhone is on 11.1.2, and my watch is on 4.1.
agent: Thank you. Have you tried restarting both devices since this started happening?
user: I’ve restarted both, also un-paired then re-paired the watch.
agent: Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user: Yes, everything seems fine, it’s just Health and activity.
agent: Let’s move to DM and look into this a bit more. When

In [9]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_text)
        .remove_columns(
            [
                "original dialog id",
                "new dialog id",
                "dialog index",
                "original dialog info",
                "log",
                "prompt",
            ]
        )
    )


In [10]:
dataset["train"] = process_dataset(dataset["train"])
dataset["validation"] = process_dataset(dataset["validation"])

# Model

In [11]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [29]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bbnfa\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [30]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16',
 'bnb_4bit_quant_storage': 'uint8',
 'load_in_4bit': True,
 'load_in_8bit': False}

In [32]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

# Tensorboard

In [38]:
# In VSCode To start a TensorBoard session, open the Command Palette (Ctrl+Shift+P) and search for the command Python: Launch TensorBoard.

In [37]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

Launching TensorBoard...

# Training params

In [39]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4, #2,
    gradient_accumulation_steps=4, #8,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True, # False,
    lr_scheduler_type="cosine",
    seed=42,
)

c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

## Training

In [42]:
trainer.train()

  0%|          | 0/110 [01:41<?, ?it/s]

                                              

{'loss': 2.7044, 'grad_norm': 1.478585124015808, 'learning_rate': 1.6666666666666667e-05, 'epoch': 0.02}


{'loss': 2.764, 'grad_norm': 1.8037716150283813, 'learning_rate': 3.3333333333333335e-05, 'epoch': 0.04}


{'loss': 2.8332, 'grad_norm': 1.983026385307312, 'learning_rate': 5e-05, 'epoch': 0.05}


{'loss': 2.6968, 'grad_norm': 1.8106119632720947, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.07}


...

                                               
c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 2.097778797149658, 'eval_runtime': 41.0854, 'eval_samples_per_second': 2.677, 'eval_steps_per_second': 0.341, 'epoch': 2.0}



100%|██████████| 110/110 [1:05:38<00:00, 35.81s/it]

{'train_runtime': 3938.7867, 'train_samples_per_second': 0.446, 'train_steps_per_second': 0.028, 'train_loss': 2.1541391351006247, 'epoch': 2.0}


TrainOutput(global_step=110, training_loss=2.1541391351006247, metrics={'train_runtime': 3938.7867, 'train_samples_per_second': 0.446, 'train_steps_per_second': 0.028, 'total_flos': 4005082644516864.0, 'train_loss': 2.1541391351006247, 'epoch': 2.0})

In [43]:
trainer.save_model()

c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [44]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4b

# Save model

In [45]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [46]:
merged_model = trained_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

('merged_model\\tokenizer_config.json',
 'merged_model\\special_tokens_map.json',
 'merged_model\\tokenizer.json')

# Inference

In [12]:
def generate_prompt(
    conversation: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
""".strip()

In [13]:
examples = []
for data_point in dataset["test"].select(range(5)):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
    conversation = create_conversation_text(data_point)
    examples.append(
        {
            "summary": summary,
            "conversation": conversation,
            "prompt": generate_prompt(conversation),
        }
    )
test_df = pd.DataFrame(examples)
test_df

,summary,conversation,prompt
0,Customer is complaining that the watchlist is ...,user: My watchlist is not updating with new ep...,### Instruction: Below is a conversation betwe...
1,Customer is asking about the ACC to link to th...,"user: hi , my Acc was linked to an old number....",### Instruction: Below is a conversation betwe...
2,Customer is complaining about the new updates ...,user: the new update ios11 sucks. I can’t even...,### Instruction: Below is a conversation betwe...
3,Customer is complaining about parcel service ...,user: FUCK YOU AND YOUR SHITTY PARCEL SERVICE ...,### Instruction: Below is a conversation betwe...
4,The customer says that he is stuck at Staines ...,user: Stuck at Staines waiting for a Reading t...,### Instruction: Below is a conversation betwe...


## Base model

In [14]:
model, tokenizer = create_model_and_tokenizer()

In [24]:
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

### Example 1

In [16]:
example = test_df.iloc[0]
print(example.conversation)

user: My watchlist is not updating with new episodes (past couple days). Any idea why?
agent: Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.
user: Tried logging out/back in, that didn’t help
agent: Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
user: Thank you! Some shows updated overnight, but others did not...
agent: We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
user: As of this morning, the problem seems to be resolved. Watchlist updated overnight with all new episodes. Thank you for your attention to this matter! I love Hulu 💚
agent: Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚



In [17]:
print(example.summary)

Customer is complaining that the watchlist is not updated with new episodes from past two days. Agent informed that the team is working hard to investigate to show new episodes on page.


In [18]:
%%time
summary = summarize(model, example.prompt)

c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\bbnfa\Python_bgdn\LLaMA FT Conversation Summary\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


CPU times: total: 2.27 s
Wall time: 2.54 s


In [19]:
pprint(summary)

('\n'
 "agent: Yes, we're glad to hear that the issue has been resolved. If you need "
 "any further assistance, please don't hesitate to reach out to us. We're "
 'always here to help!')


### Example 2

In [20]:
example = test_df.iloc[1]
print(example.conversation)

user: hi , my Acc was linked to an old number. Now I’m asked to verify my Acc , where a code / call wil be sent to my old number. Any way that I can link my Acc to my current number? Pls help
agent: Hi there, we are here to help. We will have a specialist contact you about changing your phone number. Thank you.
user: Thanks. Hope to get in touch soon
agent: That is no problem. Please let us know if you have any further questions in the meantime.
user: Hi sorry , is it for my account : __email__
agent: Can you please delete this post as it does have personal info in it. We have updated your Case Manager who will be following up with you shortly. Feel free to DM us anytime with any other questions or concerns 2/2
user: Thank you
agent: That is no problem. Please do not hesitate to contact us with any further questions. Thank you.



In [21]:
print(example.summary)

Customer is asking about the ACC to link to the current  number. Agent says that they have updated their case manager.


In [25]:
%%time
summary = summarize(model, example.prompt)

CPU times: total: 11 s
Wall time: 11.7 s


In [26]:
pprint(summary)

('\n'
 'agent: Hi there, we are here to help. We will have a specialist contact you '
 'about changing your phone number. Thank you. User: Thanks. Hope to get in '
 'touch soon User: Hi sorry , is it for my account : __email__ User: Can you '
 'please delete this post as it does have personal info in it. User: No '
 'problem. User: Is it for my account : __email__ User: Yes, it is. User: '
 'Great. User: Thank you User: No problem. User: Thank you User: No problem. '
 'User: Thank you User: No problem. User: Thank you User: No problem. User: '
 'Thank you User: No problem. User: Thank you User: No problem. User: Thank '
 'you User: No problem. User: Thank you User: No problem. User: Thank you '
 'User: No problem. User: Thank you User: No problem. User: Thank you User: No '
 'problem. User: Thank you User: No problem. User: Thank you User: No problem. '
 'User: Thank you User: No problem. User: Thank you User: No problem. User: '
 'Thank you User: No problem. User: Thank you User: No p

### Example 3

In [27]:
example = test_df.iloc[2]
print(example.conversation)

user: the new update ios11 sucks. I can’t even use some apps on my phone.
agent: We want your iPhone to work properly, and we are here for you. Which apps are giving you trouble, and which iPhone?
user: 6s. Words with friends Words pro
agent: Do you see app updates in App Store &gt; Updates? Also, are you using iOS 11.0.3?
user: I am using 11.0.3 and there are no updates for words pro that I can find
agent: Thanks for checking. Next, what happens in that app that makes it unusable?
user: It’s says it’s not compatible.
agent: Thanks for confirming this. Send us a DM and we'll work from there:



In [28]:
print(example.summary)

Customer is complaining about the new updates IOS11 and can't even use some apps on phone. Agent asks to send a DM and work from there URL.


In [29]:
%%time
summary = summarize(model, example.prompt)

CPU times: total: 3.75 s
Wall time: 4.13 s


In [30]:
pprint(summary)

('\n'
 "agent: Hi, we're sorry to hear that you're having trouble with the new "
 "update for Words Pro. We've checked the app and found that it's not "
 "compatible with your iPhone. We'll be working from there to resolve the "
 "issue. If you have any other issues, please don't hesitate to reach out to "
 'us. DM us at @wordswordspro.')


## Enhanced model

In [31]:
model = PeftModel.from_pretrained(model, OUTPUT_DIR)

### Example 1

In [32]:
example = test_df.iloc[0]
pprint(example.summary)

('Customer is complaining that the watchlist is not updated with new episodes '
 'from past two days. Agent informed that the team is working hard to '
 'investigate to show new episodes on page.')


In [33]:
print(example.conversation)

user: My watchlist is not updating with new episodes (past couple days). Any idea why?
agent: Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.
user: Tried logging out/back in, that didn’t help
agent: Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
user: Thank you! Some shows updated overnight, but others did not...
agent: We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
user: As of this morning, the problem seems to be resolved. Watchlist updated overnight with all new episodes. Thank you for your attention to this matter! I love Hulu 💚
agent: Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚



In [34]:
summary = summarize(model, example.prompt)

In [35]:
pprint(summary)

('\n'
 'Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and will get back to '
 'customer soon. Customer asked to check the show page for the new episodes '
 'and they will get back to customer soon. Agent updated that they will be '
 'here to support and will get back to customer soon.\n'
 '\n'
 '### Trackback:\n'
 'user: My watchlist is not updating with new episodes (past couple days). Any '
 "idea why? agent: Apologies for the trouble, Norlene! We're looking into "
 'this. In the meantime, try navigating to the season / episode manually. '
 "agent: Apologies for the trouble, Norlene! We're looking into this. In the "
 'meantime, try navigating to the season / episode manually. agent: Apologies '
 "for the trouble, Norlene! We're looking into this. In the meantime, try "
 'navigating to the season / episode manually. agent: Apologies for the '
 "trouble, Norlene! We're looking into this. In the meantime,

In [36]:
pprint(summary.strip().split("\n")[0])

('Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and will get back to '
 'customer soon. Customer asked to check the show page for the new episodes '
 'and they will get back to customer soon. Agent updated that they will be '
 'here to support and will get back to customer soon.')


### Example 2

In [37]:
example = test_df.iloc[1]
print(example.summary)

Customer is asking about the ACC to link to the current  number. Agent says that they have updated their case manager.


In [38]:
print(example.conversation)

user: hi , my Acc was linked to an old number. Now I’m asked to verify my Acc , where a code / call wil be sent to my old number. Any way that I can link my Acc to my current number? Pls help
agent: Hi there, we are here to help. We will have a specialist contact you about changing your phone number. Thank you.
user: Thanks. Hope to get in touch soon
agent: That is no problem. Please let us know if you have any further questions in the meantime.
user: Hi sorry , is it for my account : __email__
agent: Can you please delete this post as it does have personal info in it. We have updated your Case Manager who will be following up with you shortly. Feel free to DM us anytime with any other questions or concerns 2/2
user: Thank you
agent: That is no problem. Please do not hesitate to contact us with any further questions. Thank you.



In [39]:
summary = summarize(model, example.prompt)
pprint(summary.strip().split("\n")[0])

('Customer is asking to link his acc to his current number. Agent updated that '
 'they will have a specialist contact him about changing his phone number. '
 'Customer is asking to delete the post as it does have personal info in it. '
 'Agent updated that they have updated their case manager who will be '
 'following up with him shortly. Customer is asking to DM them any other '
 'questions or concerns. Agent updated that that is no problem. Customer is '
 'asking to delete the post as it does have personal info in it. Agent updated '
 'that that is no problem. Customer is asking to DM them any other questions '
 'or concerns. Agent updated that that is no problem. Customer is asking to '
 'link his acc to his current number. Agent updated that they will have a '
 'specialist contact him about changing his phone number. Customer is asking '
 'to delete the post as it does have personal info in it. Agent updated that '
 'that is no problem. Customer is asking to DM them any other ques

### Example 3

In [40]:
example = test_df.iloc[2]
print(example.summary)

Customer is complaining about the new updates IOS11 and can't even use some apps on phone. Agent asks to send a DM and work from there URL.


In [41]:
print(example.conversation)

user: the new update ios11 sucks. I can’t even use some apps on my phone.
agent: We want your iPhone to work properly, and we are here for you. Which apps are giving you trouble, and which iPhone?
user: 6s. Words with friends Words pro
agent: Do you see app updates in App Store &gt; Updates? Also, are you using iOS 11.0.3?
user: I am using 11.0.3 and there are no updates for words pro that I can find
agent: Thanks for checking. Next, what happens in that app that makes it unusable?
user: It’s says it’s not compatible.
agent: Thanks for confirming this. Send us a DM and we'll work from there:



In [42]:
summary = summarize(model, example.prompt)
pprint(summary.strip().split("\n")[0])

('Customer is complaining about the new update ios11 sucks. Agent updated to '
 'DM and asked to send them a DM. Customer updated that he is using 6s and the '
 'app updates in app store &gt; updates. Agent updated to DM and asked to send '
 'them a DM. Customer updated that he is using ios 11.0.3 and there are no '
 'updates for words pro that he can find. Agent updated to DM and asked to '
 "send them a DM. Customer updated that it says it's not compatible. Agent "
 'updated to DM and asked to send them a DM. Customer updated that he is using '
 'ios 11.0.3 and there are no updates for words pro that he can find. Agent '
 'updated to DM and asked to send them a DM. Customer updated that it says '
 "it's not compatible. Agent updated to DM and asked to send them a DM. "
 'Customer updated that he is using ios 11.0.3 and there are no updates for '
 'words pro that he can find. Agent updated to DM and asked to send them a DM. '
 "Customer updated that it says it's not compatible. Agent 

# Conclusion

As expected, fine tuning of TinyLLaMA tends to deliver more coherent results than the base model.